# Vector Only RAG

In this notebook we are going to explore how to make a vector only RAG system. RAG stands for Retrieval Augmented Generation, the basic idea of this technique is to not use the LLM to recall knowledge but instead rely on a external system like a database to provide information to the system.

The main advantage of vector-only RAG systems over Knowledge Graph RAG is the fact that you don't need to model your knowledge domain and can use very simple data pipelines. They excel in retrieving information based on the context of the question but fail short in retrieving factual knowledge (you've been warned).

In this tutorial we are going to present you a small system that is nice to learn how RAG works in practice. To make things bit more complicated, we are going to start from a PDF file.

The PDF we are going to parse is a paper from [Elisabeth Spelke](https://en.wikipedia.org/wiki/Elizabeth_Spelke) a developmental psychologist that studied cognition in infants, culturally different communities and non-human species to understand the fundamental building blocks of our mind. It contains evidences that our cognition system is build upon several key components and is an important inspiration for Neuro-Symbolic and Robotics systems.

In [ ]:
from hybridagi.core.pipeline import Pipeline
from hybridagi.embeddings import SentenceTransformerEmbeddings
from hybridagi.modules.splitters import DocumentSentenceSplitter
from hybridagi.modules.embedders import DocumentEmbedder
from hybridagi.readers import PDFReader

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384, # The dimention of the embeddings vector (also called dense vector)
)

reader = PDFReader()

input_docs = reader("../data/SpelkeKinzlerCoreKnowledge.pdf")

# Now that we have our input documents, we can start to make our data processing pipeline

pipeline = Pipeline()

pipeline.add("chunk_documents", DocumentSentenceSplitter(
    method = "word",
    chunk_size = 1024,
    chunk_overlap = 0,
))
pipeline.add("embed_chunks", DocumentEmbedder(embeddings=embeddings))

output_docs = pipeline(input_docs)

#### Saving the documents into memory

Now that we have our documents we can load them into memory, for storing unstructured textual documents, we provide the `DocumentMemory` and for this example we are going to use the local integration for rapid prototyping.

In [8]:
from hybridagi.memory.integration.falkordb import FalkorDBDocumentMemory

document_memory = FalkorDBDocumentMemory(
    index_name="vector_rag_agent",
    embeddings=embeddings,
    wipe_on_start=True,
)

document_memory.update(input_docs)
document_memory.update(output_docs)

### Making a RAG Agent

Now that our data is ready and loaded into memory, we can start making our RAG Agent, but first we need to create our graph program that is going to encode the Agent behavior.

In [ ]:
import hybridagi.core.graph_program as gp

# We first need to program our RAG Agent using Graph Prompt Programs
# Here we a simple program that involve 2 steps
# We first retrieve documents, then answer to the objective's question based on them

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Action(
    id = "document_search",
    purpose = "Find relevant documents",
    tool = "DocumentSearch",
    prompt = "Please infer the similarity search query (only ONE item) based on the Objective's question",
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer the Objective's question",
    tool = "Speak",
    prompt = """
Please answer the Objective's question using the relevant documents in your context.
If no document are relevant just say that you don't know.
Don't state the Objective's question and only give the correct answer.
""",
))

main.connect("start", "document_search")
main.connect("document_search", "answer")
main.connect("answer", "end")

main.build() # Verify that the graph program is correct

print(main) # Let's look at it


In [11]:
# Now we can add this program into memory

from hybridagi.memory.integration.falkordb import FalkorDBProgramMemory

program_memory = FalkorDBProgramMemory(
    index_name="vector_rag_agent",
    embeddings=embeddings,
)

program_memory.update(main)

In [ ]:
# Then instanciate our tools and agent system

import dspy
from hybridagi.core.datatypes import AgentState, Query
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.retrievers.integration.local import FAISSDocumentRetriever
from hybridagi.modules.agents.tools import (
    SpeakTool,
    DocumentSearchTool,
)

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state
    ),
    DocumentSearchTool(
        retriever = FAISSDocumentRetriever(
            document_memory = document_memory,
            embeddings = embeddings,
            distance = "cosine",
            max_distance = 0.7,
            k = 5,
            reranker = None,
        ),
    ),
]

rag_agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)

# We can now setup the LLM using Ollama client from DSPy

lm = dspy.OllamaLocal(
    model='mistral', 
    max_tokens=1024, 
    stop=["\n\n\n", "\n---", "\n\nContext:"]
)
dspy.configure(lm=lm)

# And call our agent

result = rag_agent(Query(query="What are the core knowledge?"))

print(result.final_answer)